In [2]:
import sqlite3
import pandas as pd
import numpy as np

# 1. Connect to an in-memory database
# (Use 'indian_ratings.db' for a file-based database that persists)
conn = sqlite3.connect('indian_ratings.db')
cursor = conn.cursor()

# 2. Drop the table if it already exists to start fresh
cursor.execute('DROP TABLE IF EXISTS indian_ratings')

# 3. Create a new table with the PRIMARY KEY constraint
cursor.execute('''
CREATE TABLE indian_ratings (
    user_id TEXT,
    movie_id TEXT,
    rating INTEGER,
    PRIMARY KEY (user_id, movie_id)
);
''')
conn.commit()

# 4. Generate sample data
users = ['Rahul', 'Priya', 'Amit', 'Pooja', 'Sanjay', 'Deepika', 'Akash', 'Shruti', 'Vikram', 'Anjali']
movies = ['Sholay', '3 Idiots', 'Dangal', 'Baahubali 2', 'Lagaan', 'Gully Boy', 'Bajirao Mastani', 'RRR', 'K.G.F: Chapter 2']
np.random.seed(42)

data_to_insert = []
# Ensure a sufficient number of unique ratings
while len(data_to_insert) < 50:
    user = np.random.choice(users)
    movie = np.random.choice(movies)
    rating = np.random.randint(1, 6) # Ratings from 1 to 5

    # Check for uniqueness before adding to the list
    if (user, movie) not in [(d[0], d[1]) for d in data_to_insert]:
        data_to_insert.append((user, movie, rating))

# 5. Insert the generated data into the table
cursor.executemany('INSERT INTO indian_ratings VALUES (?, ?, ?)', data_to_insert)
conn.commit()

print(f"Database created and populated with {len(data_to_insert)} ratings.")

# Close the connection for now
conn.close()

Database created and populated with 50 ratings.


In [3]:
import sqlite3
import pandas as pd

# Re-open the connection to the database file created in Part 1
conn = sqlite3.connect('indian_ratings.db')

# 1. Query all ratings into a pandas DataFrame
query = "SELECT user_id, movie_id, rating FROM indian_ratings"
long_format_ratings = pd.read_sql_query(query, conn)

print("Ratings loaded from database:")
print(long_format_ratings.head())
print(f"Total entries: {len(long_format_ratings)}")

# 2. Save the DataFrame to a CSV file
csv_file_path = 'indian_ratings.csv'
long_format_ratings.to_csv(csv_file_path, index=False)

print(f"\nData successfully saved to '{csv_file_path}'")

# Close the database connection
conn.close()

Ratings loaded from database:
   user_id         movie_id  rating
0    Akash      Baahubali 2       5
1   Shruti           Lagaan       2
2     Amit  Bajirao Mastani       3
3   Shruti              RRR       3
4  Deepika           Lagaan       2
Total entries: 50

Data successfully saved to 'indian_ratings.csv'


In [4]:
import pandas as pd
import numpy as np

# Load the data from the CSV file
csv_file_path = 'indian_ratings.csv'
long_format_ratings = pd.read_csv(csv_file_path)

print("Loaded ratings from CSV (long format):")
print(long_format_ratings.head())

# Pivot the data to create a user-item matrix with movies as rows and users as columns
ratings_matrix = long_format_ratings.pivot(index='movie_id', columns='user_id', values='rating').fillna(0)

print("\nUser-Item Matrix (items as rows):")
print(ratings_matrix.head())

Loaded ratings from CSV (long format):
   user_id         movie_id  rating
0    Akash      Baahubali 2       5
1   Shruti           Lagaan       2
2     Amit  Bajirao Mastani       3
3   Shruti              RRR       3
4  Deepika           Lagaan       2

User-Item Matrix (items as rows):
user_id          Akash  Amit  Anjali  Deepika  Pooja  Priya  Rahul  Sanjay  \
movie_id                                                                     
3 Idiots           4.0   0.0     0.0      0.0    4.0    2.0    2.0     0.0   
Baahubali 2        5.0   0.0     4.0      3.0    5.0    0.0    0.0     0.0   
Bajirao Mastani    5.0   3.0     4.0      5.0    4.0    0.0    0.0     2.0   
Dangal             0.0   0.0     4.0      0.0    0.0    0.0    0.0     1.0   
Gully Boy          0.0   4.0     5.0      0.0    5.0    2.0    5.0     0.0   

user_id          Shruti  Vikram  
movie_id                         
3 Idiots            1.0     2.0  
Baahubali 2         0.0     5.0  
Bajirao Mastani     0.0    

In [6]:
import pandas as pd
import numpy as np

# Assuming ratings_matrix from Part 1 is already loaded and prepared

# Compute the item-item similarity matrix.
# We will use pandas.corr() with method='pearson', which is equivalent to
# cosine similarity for uncentered data.
item_similarity_matrix = ratings_matrix.T.corr(method='pearson')

print("\nItem-Item Cosine Similarity Matrix:")
print(item_similarity_matrix.head())


Item-Item Cosine Similarity Matrix:
movie_id         3 Idiots  Baahubali 2  Bajirao Mastani    Dangal  Gully Boy  \
movie_id                                                                       
3 Idiots         1.000000     0.498922         0.068041 -0.421076   0.046575   
Baahubali 2      0.498922     1.000000         0.633518  0.207613  -0.114819   
Bajirao Mastani  0.068041     0.633518         1.000000  0.094547  -0.076056   
Dangal          -0.421076     0.207613         0.094547  1.000000   0.231415   
Gully Boy        0.046575    -0.114819        -0.076056  0.231415   1.000000   

movie_id         K.G.F: Chapter 2    Lagaan       RRR    Sholay  
movie_id                                                         
3 Idiots                 0.185907 -0.232175  0.459432 -0.280900  
Baahubali 2              0.017856 -0.332911  0.514533 -0.519370  
Bajirao Mastani         -0.553699 -0.264044  0.123791 -0.589768  
Dangal                  -0.115287 -0.427366 -0.246075 -0.081107  
Gully 

In [9]:
import pandas as pd
import numpy as np

# Assuming ratings_matrix and item_similarity_matrix are already available

def get_k_similar_items(item_sim_matrix, item, k=5):
    """
    Finds the k most similar items for a given item.
    """
    sim_scores = item_sim_matrix[item].drop(item)
    return sim_scores.sort_values(ascending=False).head(k)

def recommend_items_item_based(ratings_matrix, item_sim_matrix, user, k=5):
    """
    Generates recommendations for a user based on item-based collaborative filtering.
    """
    # 1. Get the user's rated movies
    user_ratings = ratings_matrix[user][ratings_matrix[user] > 0]

    # 2. Find similar items for each of the user's rated movies
    weighted_scores = pd.Series(dtype=float)
    for movie, rating in user_ratings.items():
        similar_items = get_k_similar_items(item_sim_matrix, movie, k)

        # Multiply the similarity score by the user's rating for that movie
        weighted_scores = weighted_scores.add(similar_items * rating, fill_value=0)

    # 3. Filter out movies the user has already rated and sort by score
    recommendations = weighted_scores.drop(user_ratings.index, errors='ignore').sort_values(ascending=False)

    return recommendations

# Example: Get recommendations for a random user
random_user = np.random.choice(ratings_matrix.columns)
print(f"\nRecommendations for {random_user}:")
recommendations = recommend_items_item_based(ratings_matrix, item_similarity_matrix, random_user, k=5)
print(recommendations)


Recommendations for Priya:
movie_id
RRR                2.073395
Baahubali 2        0.857488
Dangal             0.462831
Bajirao Mastani   -0.016029
Sholay            -0.041718
dtype: float64
